# Prueba Solemne N°2 - Forma B
**Taller de Programación II**

Objetivo: aplicar técnicas de limpieza, transformación y visualización de datos en Python usando el dataset `material-prueba/Forma_B.csv`.

## 1. Carga y exploración inicial
Se carga el archivo CSV y se revisan la estructura y las estadísticas básicas del conjunto de datos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid')

In [ ]:
ruta = 'material-prueba/Forma_B.csv'
try:
    df_raw = pd.read_csv(ruta)
except UnicodeDecodeError:
    df_raw = pd.read_csv(ruta, encoding='latin1')

df_raw.head()

In [ ]:
df_raw.info()

In [ ]:
df_raw.describe()

## 2. Limpieza de datos
Se imputan valores faltantes: causas de muerte con 0 y el código con 'Desconocido'.

In [ ]:
df = df_raw.copy()
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

df['code'] = df['code'].fillna('Desconocido')
# Se imputan valores faltantes en columnas de causas (numéricas)
causa_cols = [c for c in df.columns if c not in ['country', 'code', 'year']]
df[causa_cols] = df[causa_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

df.isna().sum().head(10)

## 3. Normalización y renombrado de columnas
Se renombran todas las columnas al español y se mantienen nombres en minúsculas con guiones bajos.

In [ ]:
mapa_columnas = {
    'country': 'pais',
    'code': 'codigo',
    'year': 'año',
    'meningitis': 'meningitis',
    "alzheimer's_diesease": 'enfermedad_de_alzheimer',
    "parkinson's_disease": 'enfermedad_de_parkinson',
    'nutritional_deficiency': 'deficiencia_nutricional',
    'malaria': 'malaria',
    'drowning': 'ahogamiento',
    'interpersonal_violence': 'violencia_interpersonal',
    'maternal_disorders': 'trastornos_maternos',
    'hiv/aids': 'vih_sida',
    'drug_use_disorders': 'trastornos_por_uso_de_drogas',
    'tuberculosis': 'tuberculosis',
    'cardiovascular_diseases': 'enfermedades_cardiovasculares',
    'lower_respiratory_infections': 'infecciones_respiratorias_bajas',
    'neonatal_disorders': 'trastornos_neonatales',
    'alcohol_use_disorders': 'trastornos_por_uso_de_alcohol',
    'self_harm': 'autolesiones',
    'exposure_to_forces_of_nature': 'exposicion_a_fuerzas_de_la_naturaleza',
    'diarrheal_diseases': 'enfermedades_diarreicas',
    'environmental_heat_and_cold_exposure': 'exposicion_a_calor_y_frio_ambiental',
    'neoplasms': 'neoplasias',
    'conflict_and_terrorism': 'conflicto_y_terrorismo',
    'diabetes_mellitus': 'diabetes_mellitus',
    'chronic_kidney_disease': 'enfermedad_renal_cronica',
    'poisonings': 'intoxicaciones',
    'protein_energy_malnutrition': 'desnutricion_proteico_energetica',
    'terrorism': 'terrorismo',
    'road_injuries': 'lesiones_por_transito',
    'chronic_respiratory_diseases': 'enfermedades_respiratorias_cronicas',
    'chronic_liver_diseases': 'enfermedades_hepaticas_cronicas',
    'digestive_diseases': 'enfermedades_digestivas',
    'fire_heat_hot_substance': 'fuego_calor_sustancias_calientes',
    'acute_hepatitis': 'hepatitis_aguda',
}

df = df.rename(columns=mapa_columnas)
df.columns

## 4. Conversión de tipos
Se asegura que las columnas numéricas estén en formato numérico y el año quede como entero.

In [ ]:
id_cols = ['pais', 'codigo', 'año']
causa_cols = [c for c in df.columns if c not in id_cols]

df[causa_cols] = df[causa_cols].apply(pd.to_numeric, errors='coerce').fillna(0)
df['año'] = pd.to_numeric(df['año'], errors='coerce').fillna(0).astype(int)

df.dtypes.head(10)

## 5. Cálculo de muertes totales y estadísticas
Se crea la columna `total_muertes` y se calculan media, mediana y desviación estándar.

In [ ]:
df['total_muertes'] = df[causa_cols].sum(axis=1)

estadisticas_total = df['total_muertes'].agg(['mean', 'median', 'std']).to_frame(name='valor')
estadisticas_total

## 6. Filtrado de países destacados
Se determina el percentil 75 de `total_muertes` y se identifican los países destacados.

In [ ]:
p75 = df['total_muertes'].quantile(0.75)
paises_destacados = df.loc[df['total_muertes'] >= p75, 'pais'].unique()

len(paises_destacados), paises_destacados[:10]

## 7. Análisis por año
Se calcula la mortalidad anual promedio y se identifican las principales causas y países con más muertes acumuladas.

In [ ]:
mortalidad_anual = df.groupby('año')['total_muertes'].mean().reset_index()

causa_cols_sin_total = [c for c in df.columns if c not in id_cols + ['total_muertes']]
top_causas = df[causa_cols_sin_total].sum().sort_values(ascending=False).head(5)

top_paises = (
    df.groupby('pais')['total_muertes']
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

top_causas

top_paises

## 8. Visualizaciones
Se generan los gráficos solicitados para interpretar los resultados.

In [ ]:
# KDE de total_muertes
plt.figure(figsize=(8, 5))
sns.kdeplot(df['total_muertes'], fill=True)
plt.title('Distribución (KDE) de Total de Muertes')
plt.xlabel('Total de muertes')
plt.ylabel('Densidad')
plt.tight_layout()
plt.show()

In [ ]:
# Gráfico de torta: top 5 causas
plt.figure(figsize=(7, 7))
plt.pie(top_causas.values, labels=top_causas.index, autopct='%1.1f%%', startangle=140)
plt.title('Distribución de las 5 causas más comunes')
plt.tight_layout()
plt.show()

In [ ]:
# Barras: top 5 países con más muertes
plt.figure(figsize=(8, 5))
sns.barplot(x=top_paises.values, y=top_paises.index, orient='h')
plt.title('Top 5 países con más muertes acumuladas')
plt.xlabel('Total de muertes')
plt.ylabel('País')
plt.tight_layout()
plt.show()

In [ ]:
# Línea de tendencia de total_muertes por año (promedio)
plt.figure(figsize=(9, 5))
sns.lineplot(data=mortalidad_anual, x='año', y='total_muertes', marker='o')
plt.title('Tendencia anual del total de muertes (promedio)')
plt.xlabel('Año')
plt.ylabel('Total de muertes promedio')
plt.tight_layout()
plt.show()

In [ ]:
# Comparación de las 2 causas principales a lo largo del tiempo
top2_causas = list(top_causas.index[:2])
top2_por_anio = df.groupby('año')[top2_causas].sum().reset_index()

plt.figure(figsize=(9, 5))
for causa in top2_causas:
    sns.lineplot(data=top2_por_anio, x='año', y=causa, marker='o', label=causa)
plt.title('Comparación de las 2 causas principales en el tiempo')
plt.xlabel('Año')
plt.ylabel('Muertes acumuladas')
plt.legend(title='Causa')
plt.tight_layout()
plt.show()